In [1]:
import requests
from bs4 import BeautifulSoup
import json
import os
import zipfile

# 크롤링할 URL 패턴
base_url = "https://n.news.naver.com/mnews/article/023/{:010d}"

# User-Agent 설정
headers = {"User-Agent": "Mozilla/5.0"}

# 저장할 디렉터리 생성
output_dir = "naver_news"
os.makedirs(output_dir, exist_ok=True)

# 기사 번호 범위 설정
for article_num in range(3898397, 3898467):  # 범위 수정
    url = base_url.format(article_num)
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        
        # 제목 크롤링
        title_tag = soup.select_one("h2#title_area")
        title = title_tag.text.strip() if title_tag else "제목 없음"
        
        # 본문 크롤링
        article_tag = soup.select_one("article#dic_area")
        if article_tag:
            for tag in article_tag.find_all(["strong", "em", "span"]):
                tag.decompose()
            content = article_tag.get_text(separator="\n").strip()
        else:
            content = "본문 없음"
        
        # 입력 시간 크롤링
        date_tag = soup.select_one("span.media_end_head_info_datestamp_time")
        news_date = date_tag.text.strip() if date_tag else "날짜 없음"
        
        # 카테고리 크롤링
        category_tag = soup.select_one("em.media_end_categorize_item")
        news_category = category_tag.text.strip() if category_tag else "카테고리 없음"
        
        # 기자 정보 크롤링
        reporter_tag = soup.select_one("span.byline_s")
        news_reporter = reporter_tag.text.strip() if reporter_tag else "기자 정보 없음"
        
        # JSON 데이터 생성
        news_data = {
            "sourceDataInfo": {
                "newsTitle": title,
                "newsContent": content,
                "newsCompany": "조선일보",
                "newsDate": news_date,
                "newsCategory": news_category,
                "newsReporter": news_reporter,
            }
        }
        
        # 파일로 저장
        file_path = os.path.join(output_dir, f"JO_news_{article_num:010d}.json")
        with open(file_path, "w", encoding="utf-8") as f:
            json.dump(news_data, f, ensure_ascii=False, indent=4)
    
    # 요청 실패 시 패스
    else:
        continue

# ZIP 파일로 묶기
zip_filename = "naver_news.zip"
with zipfile.ZipFile(zip_filename, "w", zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk(output_dir):
        for file in files:
            zipf.write(os.path.join(root, file), arcname=file)

print(f"크롤링 완료, {zip_filename} 파일 저장됨.")

크롤링 완료, naver_news.zip 파일 저장됨.
